In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import minmax_scale, scale
from tqdm import tqdm
import time
import implicit

c:\python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_hyper_full_valid_v2_pred"
target_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\impicit_mix_als_bpr_hyper_full_valid_v2_target"

In [3]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")
    
    return mrr_score / len(y_true)

In [4]:
%%time

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(100000), size=25000, replace=False)
val_id.sort()

with open(train_path) as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in tqdm(lines):
        tracks = line.strip().split(' ')
        if len(tracks) == 0:
            print(0)
        if len(tracks) == 1:
            print(1)
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 100000:
            break

  9%|██████▎                                                                  | 99999/1160084 [03:16<34:42, 509.09it/s]

CPU times: total: 3min 17s
Wall time: 3min 17s


In [5]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: total: 5.81 s
Wall time: 5.8 s


In [6]:
def N_opt(model_als, model_bpr, N=100, q_bpr=1):
    
    N_als = N
    N_bpr = int(N * q_bpr)
    
    userid = val_id
        
    start_time = time.time()
    rec_als = model_als.recommend(userid, user_item_val, N=N_als, filter_already_liked_items=True)
    print("\n\n--- %s seconds model_als ---" % (time.time() - start_time))

    start_time = time.time()
    rec_bpr = model_bpr.recommend(userid, user_item_val, N=N_bpr, filter_already_liked_items=True)
    print("\n--- %s seconds model_bpr ---" % (time.time() - start_time))
    
    return rec_als, rec_bpr, N_als, N_bpr

In [7]:
def optimize(rec_als,
             rec_bpr,
             N_als,
             N_bpr,
             coef_bpr=1,
             norm_all=False,
             preproc=None):
    
    if not isinstance(norm_all, bool) or preproc not in ['minmax', 'standart', None]:
        raise ValueError('wrong params')
        
    if preproc is None:
        als_score = rec_als[1]
        bpr_score = rec_bpr[1]
    else:
        if norm_all:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_als))
                bpr_score = minmax_scale(rec_bpr[1].flatten(), feature_range=(0, 1), axis=0).reshape((-1, N_bpr))
            if preproc == 'standart':
                als_score = scale(rec_als[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_als))
                bpr_score = scale(rec_bpr[1].flatten(), with_mean=True, with_std=True, axis=0).reshape((-1, N_bpr))    
        else:
            if preproc == 'minmax':
                als_score = minmax_scale(rec_als[1], feature_range=(0, 1), axis=1)
                bpr_score = minmax_scale(rec_bpr[1], feature_range=(0, 1), axis=1)   
            if preproc == 'standart':
                als_score = scale(rec_als[1], with_mean=True, with_std=True, axis=1)
                bpr_score = scale(rec_bpr[1], with_mean=True, with_std=True, axis=1)     

    result = []           
    for i in tqdm(range(rec_als[0].shape[0])):
        als_df = pd.DataFrame({'trak_id': rec_als[0][i], 'als_score': als_score[i]})
        bpr_df = pd.DataFrame({'trak_id': rec_bpr[0][i], 'bpr_score': bpr_score[i] * coef_bpr})
        df = pd.merge(als_df, bpr_df, how="outer", on='trak_id', sort=False)
        df['als_prior'] = df.als_score.where(~df.als_score.isna(), df.bpr_score)
        df['total_score'] = np.where((df.als_score.isna()) | (df.bpr_score.isna()), df.als_prior, 
                                     (df.als_score + df.bpr_score) / 2)
        final_rec = df.sort_values(by='total_score', ascending=False).trak_id.values[:100]
        result.append(' '.join(map(str, final_rec)) + '\n')
        
        
    with open(pred_path, 'w') as f:
        f.writelines(result)

    with open(target_path, 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')

    mrr = calc_score(target_path, pred_path)

    return mrr

In [8]:
model_als = implicit.als.AlternatingLeastSquares(factors=200, 
                                                 regularization=10,
                                                 alpha=40,
                                                 use_native=True,
                                                 use_cg=False,
                                                 iterations=15,
                                                 random_state=42)
print('model_als.fit:')
model_als.fit(user_item)

model_als.fit:


100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [46:33<00:00, 186.24s/it]


In [9]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=400, 
                                                     learning_rate=0.075,
                                                     regularization=0.0025,
                                                     iterations=950,
                                                     verify_negative_samples=True,
                                                     random_state=42)   
print('model_bpr.fit:')
model_bpr.fit(user_item)

model_bpr.fit:


100%|███████████████████████████████████████████████| 950/950 [39:36<00:00,  2.50s/it, train_auc=99.76%, skipped=0.79%]


In [10]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=1625, q_bpr=0.425)



--- 68.06120276451111 seconds model_als ---

--- 139.45788860321045 seconds model_bpr ---


In [11]:
optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=0.925,
           norm_all=True,
           preproc='standart')

c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:10<00:00, 356.99it/s]


MRR@100 = 0.0309


0.030893617269697724

In [12]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=1500, q_bpr=0.50)



--- 70.38340377807617 seconds model_als ---

--- 145.14897274971008 seconds model_bpr ---


In [13]:
optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=1.0,
           norm_all=True,
           preproc='standart')

c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:12<00:00, 343.76it/s]


MRR@100 = 0.0314


0.03138181957058004

In [15]:
rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=542, q_bpr=0.015726)



--- 77.82595133781433 seconds model_als ---

--- 129.54903602600098 seconds model_bpr ---


In [16]:
optimize(rec_als,
           rec_bpr,
           N_als,
           N_bpr,
           coef_bpr=0.438263,
           norm_all=True,
           preproc='minmax')

100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:04<00:00, 386.28it/s]


MRR@100 = 0.0250


0.0250455794670512

In [17]:
ls = []

In [18]:
for N in [1000, 1500, 2000]:
    for q_bpr in [0.01, 0.015, 0.1, 0.5, 1, 2]:
        rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=q_bpr)
        for norm_all in [True, False]:
            for preproc in ['minmax', 'standart']:
                for coef_bpr in [0.1, 0.5, 1, 2]:
                    print(f'N: {N}, q_bpr: {q_bpr}, norm_all: {norm_all}, preproc: {preproc}, coef_bpr: {coef_bpr}')
                    mrr = optimize(rec_als,
                                   rec_bpr,
                                   N_als,
                                   N_bpr,
                                   coef_bpr=coef_bpr,
                                   norm_all=norm_all,
                                   preproc=preproc)
                    ls.append((mrr, N, q_bpr, norm_all, preproc, coef_bpr))



--- 68.2179925441742 seconds model_als ---

--- 129.89738202095032 seconds model_bpr ---
N: 1000, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.1


100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:57<00:00, 435.91it/s]


MRR@100 = 0.0165
N: 1000, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 0.5


100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [00:57<00:00, 432.30it/s]


MRR@100 = 0.0258
N: 1000, q_bpr: 0.01, norm_all: True, preproc: minmax, coef_bpr: 1


...

c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
10

MRR@100 = 0.0316
N: 1500, q_bpr: 2, norm_all: False, preproc: standart, coef_bpr: 2


c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:239: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
10

MRR@100 = 0.0309


KeyboardInterrupt: 

In [20]:
data = pd.DataFrame(ls, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data.sort_values(by='mrr', ascending=False).head(5)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
123,0.033990,1500,0.015,False,minmax,2.0
139,0.033934,1500,0.100,False,minmax,2.0
43,0.033910,1000,0.100,False,minmax,2.0
27,0.033840,1000,0.015,False,minmax,2.0
107,0.033730,1500,0.010,False,minmax,2.0


In [31]:
ls_upd = []

In [32]:
for N in [1250, 1500, 1750, 2000]:
    rec_als, rec_bpr, N_als, N_bpr = N_opt(model_als, model_bpr, N=N, q_bpr=0.015)
    print(f'N: {N}, q_bpr: {q_bpr}, norm_all: False, preproc: minmax, coef_bpr: 3')
    mrr = optimize(rec_als,
                   rec_bpr,
                   N_als,
                   N_bpr,
                   coef_bpr=3,
                   norm_all=False,
                   preproc='minmax')
    ls_upd.append((mrr, N, 0.015, False, 'minmax', 3))



--- 64.51443576812744 seconds model_als ---

--- 136.41910886764526 seconds model_bpr ---
N: 1250, q_bpr: 0.015, norm_all: False, preproc: minmax, coef_bpr: 3


100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:07<00:00, 372.36it/s]


MRR@100 = 0.0340


--- 70.26007509231567 seconds model_als ---

--- 136.50864577293396 seconds model_bpr ---
N: 1500, q_bpr: 0.015, norm_all: False, preproc: minmax, coef_bpr: 3


100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:04<00:00, 385.15it/s]


MRR@100 = 0.0341


--- 87.69962215423584 seconds model_als ---

--- 119.51808977127075 seconds model_bpr ---
N: 1750, q_bpr: 0.015, norm_all: False, preproc: minmax, coef_bpr: 3


100%|███████████████████████████████████████████████████████████████████████████| 25000/25000 [01:08<00:00, 363.52it/s]


MRR@100 = 0.0340


KeyboardInterrupt: 

In [33]:
data_upd = pd.DataFrame(ls_upd, columns=['mrr', 'N', 'q_bpr', 'norm_all', 'preproc', 'coef_bpr'])
data_upd.sort_values(by='mrr', ascending=False).head(10)

,mrr,N,q_bpr,norm_all,preproc,coef_bpr
1,0.034119,1500,0.015,False,minmax,3
0,0.034048,1250,0.015,False,minmax,3
2,0.033964,1750,0.015,False,minmax,3
